In [216]:
import pandas as pd
import ast
import builtins
import os
from types import FunctionType

from PaladinEngine.archive.archive import Archive
from PaladinUI.paladin_cli.paladin_cli import dump_to_csv

In [217]:
# CSV_FILE_PATHS = ["caterpillar_output_sdn.csv", "caterpillar_output_sn.csv"]
CSV_FILE_PATHS = ["DiffTool/is_prime_naive.csv", "DiffTool/is_prime_square.csv"]
TEST_CSV_FILE_PATH_TEMPLATE = "DiffTool/test_{}"

In [218]:
def create_archive_from_csv(csv_path):
    dataframe = load_csv_to_dataframe(csv_path)
    archive_from_csv = Archive()
    rows = dataframe.to_records()
    for row in rows: #TODO: look for a builtin function instead of loop?
        record_key, record_value = create_record_key_value(row)
        archive_from_csv.store(record_key, record_value)
    return archive_from_csv

def load_csv_to_dataframe(csv_path):
    dataframe = pd.read_csv(csv_path)
    dataframe = dataframe.fillna('')
    return dataframe

def create_record_key_value(row):
    record_key = Archive.Record.RecordKey(int(row.container_id), row.field, row.stub_name)

    if row.rtype == 'function':
        value_of_record = row.value
        type_of_record = FunctionType
    elif row.rtype == 'list':
        value_of_record = ast.literal_eval(row.value)
        type_of_record = list
    else:
        value_of_record = getattr(builtins, row.rtype)(row.value)
        type_of_record = type(value_of_record)

    record_value = Archive.Record.RecordValue(
        record_key, type_of_record, value_of_record, row.expression,
        int(row.line_no), int(row.time), row.extra
    )
    return record_key, record_value

In [219]:
# Load the csv file into a DataFrame object
archives = []
for csv_path in CSV_FILE_PATHS:
    csv_archive = create_archive_from_csv(csv_path)
    archives.append(csv_archive)
    dump_to_csv(csv_archive, TEST_CSV_FILE_PATH_TEMPLATE.format(os.path.basename(csv_path))) #TODO: remove later

In [220]:
"""
Run:
python C:\Avital\Github\paladin_engine\PaladinUI\paladin_cli\paladin_cli.py --run --output-file output.py --csv DiffTool\is_prime_naive.csv --run-debug-server True --port 1234
C:\Avital\Github\paladin_engine\PaladinEngine\tests\test_resources\examples\is_prime\is_prime_naive.py

#TODO:
1) keys id
2) time: 0 is converted to [1,5] (line_no==85)
3) set: not supported
"""

'\nRun:\npython C:\\Avital\\Github\\paladin_engine\\PaladinUI\\paladin_cli\\paladin_cli.py --run --output-file output.py --csv DiffTool\\is_prime_naive.csv --run-debug-server True --port 1234\nC:\\Avital\\Github\\paladin_engine\\PaladinEngine\tests\test_resources\\examples\\is_prime\\is_prime_naive.py\n\n#TODO:\n1) keys id\n2) time: 0 is converted to [1,5] (line_no==85)\n3) set: not supported\n'